In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
import random

In [2]:
x_min=-1
x_max=1
w=1
steps=100
h=1
m=1
g=1
T=1
k=1
mu=1
N=10
c1=1
c2=1
c3=1
v=1
k=1
phi=1
u=1

beta=1/k*T
entropy_store=[]

grid=np.linspace(x_min,x_max, steps)
psi_0=(1/np.pi**.25)*np.exp(-0.5*grid**2)
V=.5*m*(w**2)*(grid**2)
deltax=(x_max-x_min)/steps

# Computing S_0

In [3]:
def calc_s0(psi_init, h, m, g, mu, x_max, x_min, steps,x_vec):
    
    #laplacian
    for i in range(1,len(psi_init)-1):
        laplacian=+-h**2/(2*m)*(np.conjugate(psi_init[i])*(psi_init[i+1]-2*psi_init[i]+psi_init[i-1])*psi_init[i]/((x_max-x_min)/steps))
    #print("Laplacian Term: ", laplacian)

    #Potential term
    potential_term=np.dot(V,psi_init)
    #print("Potential Term: ", potential_term)

    #Norm^4
    norm4=psi_init**4 #I feel like this needs to explicitly be something times complex conj...
    #print(norm4)
    norm4_term=integrate.simpson(norm4, x=x_vec)
    #print("Norm^4 Term: ", norm4_term)

    #Norm^2 (part of number operation)
    norm2=psi_init**2 #I feel like this needs to explicitly be something times complex conj...
    norm2_term=integrate.simpson(norm2, x=x_vec)
    #print("Norm^2 Term: ", norm2_term)

    S_0=-beta*(-laplacian+potential_term+(g/2)*norm4_term)-mu*norm2_term
    #print(S_0)

    return S_0

In [4]:
s0_init=calc_s0(psi_0, h, m, g, mu, x_max, x_min, steps,grid)
print(s0_init)

-10.523095167254333


In [5]:
# PERTURBATION
rand=random.choice([0,1])

if rand==0:
    print("Generating density perturbation...")
    psi=psi_0*(1+c1*v*np.sin(k*grid+phi))
if rand==1:
    print("Generating phase perturbation...")
    psi=psi_0*np.exp((1j*c2*v*np.sin(k*grid+phi)))

# Vary particle number
psi=(1+c3*u)*psi

Generating phase perturbation...


In [6]:
# Calculate reduced energy of perturbed field
s0_final=calc_s0(psi, h, m, g, mu, x_max, x_min, steps,grid)

In [ ]:
#Check acceptance ratio
# a=np.exp(s0_final-s0_init)
# print(a)

# if a>=1:
#     pass
#     print("a does not meet acceptance criteria. Perturbation rejected.")
#     entropy_store.append(s0_init)
# if a<1:
#     rand=random.random() #generate random number between 0,1
#     if rand<=a:
#         psi_0=psi
#         print("a meets acceptance criteria. Perturbation accepted.")
#         entropy_store.append(s0_final)
#     else:
#         pass
#         print("a meets acceptance criteria. Perturbation rejected.")
#         entropy_store.append(s0_init)

In [8]:
def check_func(s0_final, s0_init):
    a=np.exp(s0_final-s0_init)

    if a>=1:
        pass
        print("a does not meet acceptance criteria. Perturbation rejected.")
        accepted_s=s0_init
    if a<1:
        rand=random.random() #generate random number between 0,1
        if rand<=a:
            psi_0=psi
            print("a meets acceptance criteria. Perturbation accepted.")
            accepted_s=s0_final
        else:
            pass
            print("a meets acceptance criteria. Perturbation rejected.")
            accepted_s=s0_init
    return accepted_s

In [12]:
accepted_s=check_func(s0_final, s0_init)
entropy_store.append(accepted_s)

a meets acceptance criteria. Perturbation accepted.
